In [3]:
from vllm import LLM, SamplingParams

In [4]:
pretrained_model = "/home/aiops/chenxw/hfmodels/Qwen3-4B"

In [5]:
llm = LLM(
    model=pretrained_model,
    dtype="bfloat16",
    tensor_parallel_size=4,
    gpu_memory_utilization=0.85,
    max_model_len=20000,
    seed=0
)

INFO 07-10 13:50:56 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'classify', 'score', 'reward'}. Defaulting to 'generate'.
INFO 07-10 13:50:56 [config.py:1770] Defaulting to use mp for distributed inference
INFO 07-10 13:50:56 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-10 13:50:59 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/home/aiops/chenxw/hfmodels/Qwen3-4B', speculative_config=None, tokenizer='/home/aiops/chenxw/hfmodels/Qwen3-4B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_bac

2025-07-10 13:51:00,349 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
2025-07-10 13:51:00,371 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
2025-07-10 13:51:00,435 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
2025-07-10 13:51:00,640 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


WARNING 07-10 13:51:02 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7f22a9b6d180>
(VllmWorker rank=3 pid=1229) INFO 07-10 13:51:02 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[0], buffer_handle=(1, 10485760, 10, 'psm_9d7a05a8'), local_subscribe_addr='ipc:///tmp/30a64b75-2ef0-46fa-9364-c6df233fb025', remote_subscribe_addr=None, remote_addr_ipv6=False)
WARNING 07-10 13:51:02 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7f22a9b6d000>
(VllmWorker rank=2 pid=1228) INFO 07-10 13:51:02 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[0], buffer_handle=(1, 10485760, 10, 'psm_13109a5d'), local_subscribe_addr='ipc:///tmp/bbab7321-8c7a-42f6-b396-875

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(VllmWorker rank=2 pid=1228) (VllmWorker rank=1 pid=1227) INFO 07-10 13:52:07 [loader.py:458] Loading weights took 9.40 seconds
INFO 07-10 13:52:07 [loader.py:458] Loading weights took 9.44 seconds
(VllmWorker rank=3 pid=1229) INFO 07-10 13:52:08 [loader.py:458] Loading weights took 9.44 seconds
(VllmWorker rank=0 pid=1226) INFO 07-10 13:52:08 [loader.py:458] Loading weights took 9.54 seconds
(VllmWorker rank=2 pid=1228) INFO 07-10 13:52:08 [gpu_model_runner.py:1347] Model loading took 1.8947 GiB and 10.109819 seconds
(VllmWorker rank=1 pid=1227) INFO 07-10 13:52:08 [gpu_model_runner.py:1347] Model loading took 1.8947 GiB and 9.901504 seconds
(VllmWorker rank=3 pid=1229) INFO 07-10 13:52:08 [gpu_model_runner.py:1347] Model loading took 1.8947 GiB and 9.987425 seconds
(VllmWorker rank=0 pid=1226) INFO 07-10 13:52:08 [gpu_model_runner.py:1347] Model loading took 1.8947 GiB and 10.228815 seconds
(VllmWorker rank=2 pid=1228) INFO 07-10 13:52:26 [backends.py:420] Using cache directory: /hom

In [18]:
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=1.0,
    top_k=-1,
    min_p=0.0,
    max_tokens=10000,
    stop=["</answer>", "User:", "Human:", "Assistant:", "<|im_end|>", "<|endoftext|>"]
)

In [19]:
code_prompt="""<|im_start|>system\nYou are a helpful assistant help user solve problems. <|im_end|>\n<|im_start|>user\nYou need to think first then write python script. You should use input() to input and print() to output in your script. \nThis is the problem:\nBamboo Blossoms\n\nThe bamboos live for decades, and at the end of their lives, they flower to make their seeds. Dr. ACM, a biologist, was fascinated by the bamboos in blossom in his travel to Tsukuba. He liked the flower so much that he was tempted to make a garden where the bamboos bloom annually. Dr. ACM started research of improving breed of the bamboos, and finally, he established a method to develop bamboo breeds with controlled lifetimes. With this method, he can develop bamboo breeds that flower after arbitrarily specified years.\n\nLet us call bamboos that flower k years after sowing \"k-year-bamboos.\" k years after being sowed, k-year-bamboos make their seeds and then die, hence their next generation flowers after another k years. In this way, if he sows seeds of k-year-bamboos, he can see bamboo blossoms every k years. For example, assuming that he sows seeds of 15-year-bamboos, he can see bamboo blossoms every 15 years; 15 years, 30 years, 45 years, and so on, after sowing.\n\nDr. ACM asked you for designing his garden. His garden is partitioned into blocks, in each of which only a single breed of bamboo can grow. Dr. ACM requested you to decide which breeds of bamboos should he sow in the blocks in order to see bamboo blossoms in at least one block for as many years as possible.\n\nYou immediately suggested to sow seeds of one-year-bamboos in all blocks. Dr. ACM, however, said that it was difficult to develop a bamboo breed with short lifetime, and would like a plan using only those breeds with long lifetimes. He also said that, although he could wait for some years until he would see the first bloom, he would like to see it in every following year. Then, you suggested a plan to sow seeds of 10-year-bamboos, for example, in different blocks each year, that is, to sow in a block this year and in another block next year, and so on, for 10 years. Following this plan, he could see bamboo blossoms in one block every year except for the first 10 years. Dr. ACM objected again saying he had determined to sow in all blocks this year.\n\nAfter all, you made up your mind to make a sowing plan where the bamboos bloom in at least one block for as many consecutive years as possible after the first m years (including this year) under the following conditions:\n\n* the plan should use only those bamboo breeds whose lifetimes are m years or longer, and\n* Dr. ACM should sow the seeds in all the blocks only this year.\n\n\nInput\n\nThe input consists of at most 50 datasets, each in the following format.\n\nm n\n\n\nAn integer m (2 ≤ m ≤ 100) represents the lifetime (in years) of the bamboos with the shortest lifetime that Dr. ACM can use for gardening. An integer n (1 ≤ n ≤ 500,000) represents the number of blocks.\n\nThe end of the input is indicated by a line containing two zeros.\n\nOutput\n\nNo matter how good your plan is, a \"dull-year\" would eventually come, in which the bamboos do not flower in any block. For each dataset, output in a line an integer meaning how many years from now the first dull-year comes after the first m years.\n\nNote that the input of m = 2 and n = 500,000 (the last dataset of the Sample Input) gives the largest answer.\n\nSample Input\n\n\n3 1\n3 4\n10 20\n100 50\n2 500000\n0 0\n\n\nOutput for the Sample Input\n\n\n4\n11\n47\n150\n7368791\n\n\n\n\n\n\nExample\n\nInput\n\n3 1\n3 4\n10 20\n100 50\n2 500000\n0 0\n\n\nOutput\n\n4\n11\n47\n150\n7368791\n\n<|im_end|>\n<|im_start|>assistant\n"""


In [20]:
outputs = llm.generate([code_prompt]*16, sampling_params)

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
outputs

In [ ]:
import re


def extract_code(full_output):
    matches = re.findall(r"```python(.*?)```", full_output, re.DOTALL)
    if matches:
        code_output = matches[-1].strip()
    else:
        code_output = "We can not extract the code in the output. "
    return code_output

In [17]:
for o in outputs:
    out = o.outputs[0].text
    print("======code=====")
    print(extract_code(out))
    

======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
======code=====
We can not extract the code in the output. 
